In [ ]:
from decimal import Decimal

import matplotlib.pyplot as plt

from btopt.data.bar import Bar
from btopt.data.dataloader import CSVDataLoader
from btopt.data.timeframe import Timeframe
from btopt.engine import Engine
from btopt.log_config import logger_main
from btopt.order import Order
from btopt.strategy.strategy import Strategy

In [ ]:
class SimpleMovingCrossover(Strategy):
    def __init__(self, name, parameters=None, primary_timeframe=None):
        super().__init__(name, parameters, primary_timeframe)
        self.short_period = self.parameters.get("short_period", 10)
        self.long_period = self.parameters.get("long_period", 20)

        if self.primary_timeframe is None:
            self.primary_timeframe = Timeframe("1d")  # Set a default if not provided

    def on_bar(self, bar: Bar) -> None:
        symbol = bar.ticker
        data = self.get_data(symbol, self.primary_timeframe)

        if len(data.close) < self.long_period:
            return

        short_ma = data.close[-self.short_period :].mean()
        long_ma = data.close[-self.long_period :].mean()

        current_position = self.get_current_position(symbol)

        if short_ma > long_ma and current_position <= 0:
            self.buy(symbol, 1)

        elif short_ma < long_ma and current_position >= 0:
            self.sell(symbol, 1)

    def on_order_update(self, order: Order) -> None:
        logger_main.info(f"Order update: {order}")

    def on_trade_update(self, trade) -> None:
        logger_main.info(f"Trade update: {trade}")


def run_backtest():
    # Initialize the engine
    engine = Engine()

    # Load data
    symbol = "EURUSD"
    start_date = "2022-01-01"
    end_date = "2023-01-01"
    base_timeframe = "1m"
    strategy_timeframe = "1d"

    logger_main.info(f"Loading data for {symbol} from {start_date} to {end_date}")
    dataloader = CSVDataLoader(
        symbol, base_timeframe, start_date=start_date, end_date=end_date
    )

    logger_main.info("Adding data to engine")
    engine.add_data(dataloader)

    logger_main.info(f"Resampling data to {strategy_timeframe}")
    engine.resample_data(dataloader, strategy_timeframe)

    # Create and add the strategy
    strategy_params = {"short_period": 10, "long_period": 20}
    strategy = SimpleMovingCrossover(
        "SMA Crossover",
        strategy_params,
        # primary_timeframe=Timeframe(strategy_timeframe),
    )
    engine.add_strategy(strategy)

    # Set up the backtest configuration
    initial_capital = Decimal("100000")
    commission_rate = Decimal("0.001")  # 0.1% commission
    config = {
        "initial_capital": initial_capital,
        "commission_rate": commission_rate,
    }
    engine.set_config(config)

    # Run the backtest
    try:
        logger_main.info("Starting backtest")
        results = engine.run()

        # Print the results
        print("\nBacktest Results:")
        print(f"Total Return: {results['performance_metrics']['total_return']:.2f}%")
        print(f"Sharpe Ratio: {results['performance_metrics']['sharpe_ratio']:.2f}")
        print(f"Max Drawdown: {results['performance_metrics']['max_drawdown']:.2f}%")
        print(f"Win Rate: {results['performance_metrics']['win_rate']:.2f}%")
        print(f"Profit Factor: {results['performance_metrics']['profit_factor']:.2f}")
        print(f"Total Trades: {results['performance_metrics']['total_trades']}")

        # Plot the equity curve
        equity_curve = results["equity_curve"]
        equity_curve.set_index("timestamp", inplace=True)
        equity_curve["equity"].plot(title="Equity Curve")

        plt.show()
    except Exception as e:
        logger_main.error(f"Error during backtest: {e}", exc_info=True)

In [ ]:
run_backtest()